# Using APIs with Python

This Ipython notebook shows you how to use the World Bank API with the Python requests library. At the end there is a short exercise for you.

To access APIs in Python, you can use the [requests library](http://docs.python-requests.org/en/master/user/install/). 

To install, you can go in a terminal and type:
```
pip install pipenv
pipenv install requests
```

When you use the requests library, it's like putting an API url in a browser except now Python does this behind the scenes programatically and then stores the results in a variable. The classroom workspaces already have the requests library installed. Here is a demonstration of how the request library works.

In [ ]:
import requests
r = requests.get('http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD')
r.text

# Explanation of Results 

```
import requests
```

This line imports the requests library.

```
r = requests.get('http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD')
```

The line `requests.get` sends out a "get" request to the url specified in parenthesis. The html standard specifies different types of [request methods](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods) with the most common one being "get". A "get" request is generally reserved for obtaining data from a server. For the World Bank API, you'll only need to make "get" requests. An API's documentation will generally tell you what type of requests to make.

The last line of code outputs the results.
```
r.text
``` 

In this case, the output is xml data. [XML](https://www.w3schools.com/xml/xml_whatis.asp) is a language for storing and transmitting data. XML was standardized in the late 1990s. A few years later, the JSON format came along and serves a similar purpose. JSON tends to be much easier to work with and luckily, the World Bank API can return data in json format by specifying the format parameters. 

Next, you'll see how to send parameters with your get request.

# Sending Parameters with the Get Request

This url, `http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD`, obtains GDP data for Brazil. You specify the country name with the [2-character ISO code](http://www.nationsonline.org/oneworld/country_code_list.htm) and then specify the economic indicator that you want, which in this case is `NY.GDP.MKTP.CD`. 

To see all of the possible indicators, visit the [world bank indicator dashboard](https://data.worldbank.org/indicator?tab=all) or use the API address `http://api.worldbank.org/v2/indicators`.

The url syntax is specific to the World Bank API. Other APIs might be different.

When making a get request, you can add parameters to the end of the url with the following syntax:
`http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD?format=json&per_page=500&date=1990:2015`

where a question mark was added at the end of the url following by each parameter name and parameter values. The parameters are separated by the ampersand & sign. Adding parameters to a get request with the question mark ? and ampersand is the standard way for adding parameters to a get request.

With the requests library, you can send this get request in one of two ways. The first way specifies the entire url in a single string.

```
r = requests.get('http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD?format=json&per_page=500&date=1990:2015')

```

The second way separate the base url and the parameters.

```
payload = {'format': 'json', 'per_page': '500', 'date':'1990:2015'}
r = requests.get('http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD', params=payload)
```

Why use the second version? It's somewhat easier to write programs with the second version because you can easily change the values in the payload dictionary whereas the first version involves manipulating the url string.

Run the code cell below to show that both versions give the same results.

In [ ]:
r_url = requests.get('http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD?format=json&per_page=500&date=1990:2015')

payload = {'format': 'json', 'per_page': '500', 'date':'1990:2015'}
r_payload = requests.get('http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD', params=payload)

# Test if both results are the same string
r_url.text == r_payload.text

# Navigating the JSON response

Once you have the data, what can you do with it? The requests library comes with a json decoder. In order to obtain the json response, all you have to do is use the .json() method like so:

```
payload = {'format': 'json', 'per_page': '500', 'date':'1990:2015'}
r = requests.get('http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD', params=payload)
r.json()
```

Run the cell below to see the results:

In [ ]:
payload = {'format': 'json', 'per_page': '500', 'date':'1990:2015'}
r = requests.get('http://api.worldbank.org/v2/countries/br/indicators/NY.GDP.MKTP.CD', params=payload)
r.json()

The result is a list of json values, which you can iterate through to access all of the data. Every API will return data in a slightly different way, so you need to look at the data and figure out how to clean the data for your purposes. For example, in the json response, the first entry contains meta data about the results. The second entry is a list containing all of the data points. For the purposes of analyzing the data, you only need the second entry, which you can access with `r.json()[1]`. Run the code cells below to see an example of how to iterate through the data, clean the data, and then plot the results.

In [ ]:
# get the World Bank GDP data for Brazil, China and the United States
payload = {'format': 'json', 'per_page': '500', 'date':'1990:2016'}
r = requests.get('http://api.worldbank.org/v2/countries/br;cn;us/indicators/NY.GDP.MKTP.CD', params=payload)

# put the results in a dictionary where each country contains a list of all the x values and all the y values
# this will make it easier to plot the results
from collections import defaultdict
data = defaultdict(list)

for entry in r.json()[1]:
    # check if country is already in dictionary. If so, append the new x and y values to the lists
    if data[entry['country']['value']]:
        data[entry['country']['value']][0].append(int(entry['date']))
        data[entry['country']['value']][1].append(float(entry['value']))       
    else: # if country not in dictionary, then initialize the lists that will hold the x and y values
        data[entry['country']['value']] = [[],[]] 

# show the results contained in the data dictionary
for country in data:
    print(country)
    print(data[country][0])
    print(data[country][1])
    print('\n')

In [ ]:
# visualize the results with matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

# create a plot for each country
for country in data:
    plt.plot(data[country][0], data[country][1], label=country)

# label the plot
plt.title('GDP for Brazil, China, and USA 1990 to 2015')
plt.legend()
plt.xlabel('year')
plt.ylabel('GDP')
plt.show()

# Practice

Here is a practice exercise for you to try out. Go to the World Bank data website found [here](https://data.worldbank.org/indicator) and find the `Population growth (annual %)` indicator. If you click on the indicator link, you'll see the indicator symbol in the web URL (hint it starts with "SP").

Use the requests library to find the indicator values for China and India in the years 2013, 2014 and 2015. Put the results in a dictionary in this format:
{country_name: [[2013, 2014, 2015], [value_2013, value_2014, value_2015]]}


In [ ]:
# TODO: assign parameters to a dictionary called payload.
# Use the format parameter so that the request 
# returns data in the json format. 
# Use the date parameter to filter for the necessary years

payload = {}

# TODO: write the request and put the results in the r variable
r = ''

# TODO: store the results in a dictionary like in the previous example.
# where each the dictionary keys are a country and the dictionary
# values are lists of lists with the dates in one list and the values
# in another list.

If you did it correctly, you should get a dictionary with the following values:

```
{'China': [[2015, 2014, 2013],
[0.508136747291937, 0.506311591779847, 0.49370963351136]],
'India': [[2015, 2014, 2013],
[1.16752707459156, 1.18932821143382, 1.21941894433091]]})
```

For a solution, go to the file menu in the workspace, click open, and choose the 'api_solution.py' file.

# Authentication

The World Bank API does not require authentication; however, many API do require some sort of user authentication with a username and password that you need to send along with your request. The requests documentation gives the [basics of how to use authentication](http://docs.python-requests.org/en/master/user/authentication/) when sending a request.

The Twitter API, for example, requires you to set up credentials. 
But before you get bogged down by the Twitter API documentation and authentication protocols, you might want to do an internet search for Python twitter libraries. Oftentimes, for the APIs of the big internet companies, someone has [built a library](https://developer.twitter.com/en/docs/developer-utilities/twitter-libraries.html) that makes it much easier to use the API.